In [1]:
from flexipod_env import FlexipodEnv

In [2]:
env = FlexipodEnv()
env.info = True
env.reset()

array([ 1.00000000e+00, -9.11384905e-06,  5.00024855e-01, -8.66011024e-01,
        1.00000000e+00,  5.11829830e-06,  1.00000000e+00, -7.28082978e-06,
        5.00009775e-01,  8.66019785e-01,  1.00000000e+00,  2.11795236e-06,
        1.00000000e+00, -4.14019269e-06, -4.57008682e-05, -1.00000000e+00,
        1.00000000e+00, -4.59997354e-05,  1.00000000e+00, -1.42594172e-05,
        7.94331118e-06,  1.00000000e+00,  1.00000000e+00, -2.37700606e-05,
       -1.23536782e-04,  4.40606993e-04,  9.44163185e-05, -1.14735369e-04,
       -1.78918126e-04,  5.85891794e-05, -1.12484622e-05, -6.43731910e-04,
       -5.87223971e-04, -5.16686814e-05, -7.99433255e-05, -1.92791267e-04,
        3.12656011e-05, -1.06597596e-04, -2.15436430e-05,  2.75309412e-05,
        4.27984123e-05, -1.20214181e-05,  7.06170567e-06,  1.60716445e-04,
        1.52028282e-04,  2.63789789e-05,  2.28888639e-05,  6.09486633e-05,
        3.37462225e-05, -7.12042038e-06,  1.00000000e+00, -2.55730920e-05,
        1.00000000e+00,  

In [5]:
observation,reward,done,info = env.step()
info

{'t': 323.4324951171875,
 'vel_cost': 0.5999999997825384,
 'uph_cost': 1.1016417489756146e-09,
 'quad_ctrl_cost': 0.9999898881533239}